In [59]:
# -*- coding: utf-8 -*- 
import urllib , requests , sys ,string ,time
from bs4 import BeautifulSoup
from bs4 import SoupStrainer

#daito-i
#fkey作者來源key.txt
#fcomiclist結果key_daito-i.text

#輸出格式：
#daito-i
#!作者
#!總筆數
#==類別_數量_中文敘述
#條目

#comiclist常數
pnum = 30#頁顯示數量
mlink = 'http://www.daito-i.com/top/'#前綴網址

fkey = open('key.txt', 'r')
key = fkey.readline()#key=作者
fkey.close()

#檢查BOM
if '%EF%BB%BF' in urllib.quote(key):
    print 'fuck ms'

'''
link = "http://comiclist.jp/index.php?p=s&mode=ss&keyword=" + urllib.quote(key) + "&type=title"
res = requests.get(link)
'''
#當sjis輸出utf8的url
key2 = urllib.quote(key.decode('utf8').encode('sjis'))
#utf8的url翻sjis
key3 = urllib.unquote(key2.decode('sjis').encode('utf8'))
#虎穴用sjis

#print key2
#print key3
adict = { 
'mode':'search',
'page_num':'0',
'search_cat':'',
'keyword':key
}
res = requests.post("http://www.daito-i.com/top/show_unit.php", data = adict)

res.encoding =  res.apparent_encoding#亂碼處理
only_a_tags = SoupStrainer(id='contents')#縮小檢索範圍
soup = BeautifulSoup(res.text ,"lxml",  parse_only=only_a_tags)#.prettify()#prettify_縮進顯示html

#換頁
def next(page = 2):
    #http://comiclist.jp/index.php?p=s&mode=ss&type=title&keyword=%E4%B8%8A%E8%97%A4%E6%94%BF%E6%A8%B9&andor=and&maxline=30&pgn=3&pgn=1
    #&andor=and&maxline=無影響&pgn=無影響&pgn=頁
    #link = "http://comiclist.jp/index.php?p=s&mode=ss&type=title&keyword=" + urllib.quote(key) + "&andor=and&maxline=30&pgn=1&pgn=" + str(page)
    adict = { 
        'mode':'search',
        'category':'',
        'subcategory':'',
        'search_cat':'',
        'keyword':key,
        'sort':'',
        'page_num':str(page)
    }
    res = requests.post("http://www.daito-i.com/top/show_unit.php", data = adict)
    res.encoding =  res.apparent_encoding
    only_a_tags = SoupStrainer(id='contents')
    soup = BeautifulSoup(res.text ,"lxml",  parse_only=only_a_tags)#.prettify()
    return soup

#全轉半
def Q2B(ustring):
    fs='０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs='0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    for tm in ustring:
        if fs.find(tm)+1:
            tm = hs[fs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    return ustring
#半轉全
def B2Q(ustring):
    fs='０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs='0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    for tm in ustring:
        if hs.find(tm)+1:
            tm = fs[hs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    return ustring

#資料儲存
def save(sdict , check=0):
    #print listdata
    for temp in listdata:
        if sdict.get(temp):
            #fout.write(temp.encode('utf8') +sdict[temp].encode('utf8')  + '\n')
            fout.write(sdict[temp].encode('utf8')  + '\n')
    #return

#新刊
def find_1():
    #收網址
    #網址get
    #找出目錄_找出篇名
    #回報目錄篇名
    print 'find1'
    return 'a'
#單行本
def find_2(blink='http://www.daito-i.com/top/'):
    global bdict2
    bdate = ''
    res = requests.get(blink)
    res.encoding =  res.apparent_encoding
    only_a_tags = SoupStrainer(class_="rec")
    soup = BeautifulSoup(res.text ,"lxml",  parse_only=only_a_tags)
    print soup.select('strong')[0].text,soup.select('strong')[1].text#書名_作者
    #print len(soup.select('.goodTxt')),soup.select('.goodTxt')[0]#主內容
    
    nnum = str(soup.select('.goodTxt')[0]).count('<br/>')#<br/>次數
    for tm in soup.select('.goodTxt')[0].select('p'):
        if (str(tm).count('<br/>') > 5):# and str(tm).count(key):
            #print tm.text,type(tm.text)
            bdate = bdate + '(' + str(str(tm).count('<br/>')) + ')-'#單行本話數
            bdict2 = bdict2 + tm.get_text('_') + '_'
            tm = tm.get_text('\n-')
            bdate = bdate + tm + '%\n'
    #print 'find2'
    bdate = bdate[:bdate.find(u'%')+1]#去多餘
    return bdate,nnum
#雜誌類
def find_3(blink='http://www.daito-i.com/top/'):
    bdate = ''
    res = requests.get(blink)
    res.encoding =  res.apparent_encoding
    only_a_tags = SoupStrainer(class_="rec")
    soup = BeautifulSoup(res.text ,"lxml",  parse_only=only_a_tags)
    #print soup.select('strong')[0].text,soup.select('strong')[1].text#雜誌名_期號
    #print len(soup.select('.goodTxt')),soup.select('.goodTxt')[0]#主內容
    
    nnum = str(soup.select('.goodTxt')[0]).count(KEY)#作者出現次數
    for tm in soup.select('.goodTxt')[0].select('p'):
        if (str(tm).count('】') > 3) and str(tm).count(KEY):#特徵與作者
            #print tm.text,type(tm.text)
            tm=tm.text
            while tm.find(u'【') > 0:#特徵存在
                if KEY.decode('utf8') in tm[tm.find(u'【')+1:tm.find(u'】')]:#符合作者
                    listdict3.append(tm[:tm.find(u'【')])
                    bdate = bdate + tm[:tm.find(u'】')+1] + '_'#疊加
                tm=tm[tm.find(u'】')+1:]
    #print 'find3'
    return bdate,nnum

def find_x(blink='http://www.daito-i.com/top/comics/detail.php?code=9784860329488'):
#def find_x(blink='http://www.daito-i.com/top/magazine/detail.php?code=4910137590557'):
    #bdate = []
    bdate = ''
    res = requests.get(blink)
    res.encoding =  res.apparent_encoding
    only_a_tags = SoupStrainer(class_="rec")
    #class="rec"
    soup = BeautifulSoup(res.text ,"lxml",  parse_only=only_a_tags)#.prettify()
    #找出目錄_找出篇名
    #print len(soup.find_all(text=key.decode('utf8')))
    print soup.select('strong')[0].text,soup.select('strong')[1].text
    #print len(soup.select('.goodTxt')),soup.select('.goodTxt')[0]
    
    nnum = str(soup.select('.goodTxt')[0]).count(KEY)#作者出現次數
    print 'name_num:',nnum
    #print key
    #print '\n',w.count(key)#.decode('utf8'))
    #print w.count('<p>'),w.count('<br/>')
    #print len(soup.select('.goodTxt')[0].select('p'))
    #p = '】'
    #ww = ''
    print '===='
    #'''
    for tm in soup.select('.goodTxt')[0].select('p'):
        #tm = str(tm)
        #print ww.count('】')
        if (str(tm).count('】') > 3) and str(tm).count(KEY):
            #print tm.count('】')
            print tm.text,type(tm.text)
            #tm = str(tm)
            tm=tm.text
            #v = 0
            #print tm
            while tm.find(u'【') > 0:
                #tmm = str(tm)[str(tm).find('【'):str(tm).find('】')]
                #print tm
                if KEY.decode('utf8') in tm[tm.find(u'【')+1:tm.find(u'】')]:
                    print 'ok'#,tm[0:tm.find(u'【')]
                    #print len(tm)
                    #bdate.append(tm[:tm.find(u'【')])
                    bdate = bdate + tm[:tm.find(u'【')] + '_'
                    
                print tm.find(u'【'),tm.find(u'】')
                #print tm[tm.find(u'【'):tm.find(u'】')]
                tm=tm[tm.find(u'】')+1:]
                #print tm
                #print 'oo'
    #回報目錄篇名
    #'''
    print 'find3'
    return bdate,nnum

#資料處理
def findbook(soup , page = 1):
    global pnn
    a =0
    #while len(soup.select('.itmBox')[a]) > 3:
    for itmBox in soup.select('.itmBox'):
        #itmBox = soup.select('.itmBox')[a]
        #print 'aa'+soup.select('.itmBox')[a].text+'bb'
        #if soup.select('.itmBox')[a].select('a')[0].get('href'):
            #print 'ok'
        ctype = soup.select('.marks')[a].next_sibling[1:]
        cbook = soup.select('.itmBox')[a].select('a')[1].text
        cname = soup.select('.itmBox')[a].select('a')[2].text
        
        #作品網址
        blink = ''
        blink = soup.select('.itmBox')[a].select('a')[0].get('href')
        blink = mlink + blink
        #print blink
        
        #書名
        #print cbook,cname
        #print type(itmBox),itmBox
        
        #類型
        #print ctype
        bdate = ''#次層頁面主資料
        
        b = str(pnn).rjust(3,'0')#倒數的流水號
        #予約商品_新刊
        if ctype in u'予約商品':#予約商品_
            #print pnn,ctype,cbook,cname
            !bdate=find_1()
            book = cname + '_' + cbook + '_\n!' + blink
            dict1.setdefault(pnn,book)
        #コミックス_單行本
        elif ctype in u'コミックス':
            if KEY.decode('utf8') in itmBox.text:
                #print pnn,ctype,cbook,cname
                cfind=find_2(blink)#目標作者的單行本
                print cfind[1]
                bdate = cfind[0]
                if len(bdate) < 10:
                    bdate = 'lost'
                book = cbook + '_\n' + bdate + '\n!' + blink
                dict2.setdefault(pnn,book)
            elif cname in u'アンソロジー ':
                #print ctype,cbook,cname,'==單>雜'
                cfind=find_3(blink)#非作者or合本_アンソロジー 
                print cfind[1],'num'
                bdate=cfind[0]
                book = cname + '_' + cbook + '_' + bdate + '_\n!' + blink
                dict2.setdefault(pnn,book)
            else:
                #print ctype,cbook,cname
                book = ctype + '_' + cname + '_' + cbook + '_\n!' + blink
                dict2.setdefault(pnn,book)
        #雑誌_雜誌單篇
        elif ctype in u'雑誌':
            print ctype,cbook,cname
            cfind=find_3(blink)
            print cfind[1],'name_num'
            bdate=cfind[0]
            book = cbook + cname + '_' + bdate + '_\n!' + blink
            dict3.setdefault(pnn,book)
        #ノベルズ_文庫_画集_書籍_
        elif ctype in [u'ノベルズ',u'文庫',u'画集',u'書籍']:
            #print ctype,cbook,cname
            book = ctype + '_' + cname + '_' + cbook + '_\n!' + blink
            dict4.setdefault(pnn,book)
        #ムック_同人誌_販促品_
        elif ctype in [u'ムック',u'同人誌',u'販促品']:
            #print ctype,cbook,cname
            book = ctype + '_' + cname + '_' + cbook + '_\n!' + blink
            dict5.setdefault(pnn,book)
        
        #
        listdata.append(pnn)
        '''
        1
        予約商品_
        2
        コミックス_
        3
        雑誌_
        4
        ノベルズ_文庫_画集_書籍_
        5
        ムック_同人誌_販促品_
        '''
        
        #print soup.select('.itmBox')[a].br
            
        #print soup.select('.marks')[2].next_element
        #print itmBox.br
        
            
        '''#
        cname = soup.select('.list-name')[a].text
        cbook = soup.select('strong')[a].text
        #print soup.select('strong')[a]
        cdata = soup.select('.list-day')[a].text
        
        #日期處理
        cdata = cdata.rstrip()
        if len(cdata) < 8:#無日期
            cdata = '0000/00/00'#填入日期
            check = 4#新作
        data = cdata[:10]
        while listdata.count(data):#重複日期判斷
            data = data[:8] + str(int(data[8:]) + 1).rjust(2,'0')#日期+1_十位數填0
        listdata.append(data)
        
        #字典新增
        if check > 0:
            if check == 1:
                book = book + '_' + name + '\n' + mlink +blink#書名+作者+網址
                dict1.setdefault(data,book)
            elif check ==2:
                dict2.setdefault(data,book)
            elif check ==3:
                dict3.setdefault(data,book)
            elif check ==4:
                dict4.setdefault(data,book)
        '''#
            
        a = a + 1
        
        #特徵itmBox開滿每頁顯示才放資料，會跑出空值，以搜尋筆數跳出。
        pnn = pnn -1
        #print a,pnn,int(pnn)
        if pnn == 0:
            break
        
    print '========'

########
KEY = B2Q(key)#.decode('utf8'))
#KEY= KEY.encode('utf8')
#KEY = KEY.decode('utf8')

#if 
pn =''
pn = soup.select('tr')[1].select('td')[1].text
#pnn = int(pn)
#print pn.find(u'件')
pn = pn[0:pn.find(u'件')]#筆數
print pn

#資料筆數_是否數字
if pn > 0:
    if int(pn) > pnum:
        print 'BIG'
    
    fout = open(key.decode('utf8') + '_daito-i.txt', 'w')#寫入模式開檔
    fout.write('daito-i\n')#comiclist
    print key.decode('utf8') , pn , 'num\n========'
    time.sleep(1)
    fout.write('!' + key + '\n!總筆數' + pn.encode('utf8') + '\n')
    
    p = 0#頁
    #建空輸出用字典與陣列
    #單行本_雜誌單篇_新刊_其它
    dict1={}#新刊
    dict2={}#單行本
    dict3={}#雜誌
    dict4={}#
    dict5={}#其他
    listdict3=[]#雜誌單篇
    bdict2 = ''#單行本書目
    listdata = []
    pnn = int(pn)#計步
    
    #資料處理
    while (int(pn) - p * pnum) > 0:
        
        print 'page:' + str(p)
        soup = next(p)#頁
        findbook(soup)#資料處理
        p = p + 1
        time.sleep(1)
    
    #日期排序
    listdata.sort()
    #print '======'
    
    #sys.exit()################
    temp = ''
    #dict1_雜誌輸出
    fout.write('==new_' + str(len(dict1)) +'_新刊\n')
    save(dict1)
    #dict2_單行本輸出
    fout.write('==book_' + str(len(dict2)) +'_單行本\n')
    save(dict2)
    #dict3_雜誌輸出
    fout.write('==adult_' + str(len(dict3)) +'_雜誌\n')
    save(dict3)
    #dict4_作畫擔任輸出
    fout.write('==art_' + str(len(dict4)) +'_作畫擔任\n')
    save(dict4)
    #dict5_其他輸出
    fout.write('==other_' + str(len(dict5)) +'_其他\n')
    save(dict5)
    
    fout.write('==fd32_' + str(len(listdict3)) +'_比對結果\n')
    tmp = ''
    out32 = ''
    #listdict3_bdict2
    for tmp in listdict3:
        #print type(tmp),type(bdict2)
        if tmp in bdict2:
            #print 'catch'
            tmp='-'+tmp
        out32=out32+tmp+'\n'
    fout.write(out32.encode('utf8'))
    
    fout.close()
    print 'ok'
elif p:
    print '同人'

#結束讀秒
x=3
while x!=0:
    print x,'..',
    x=x-1
    time.sleep(1)
print 'end'


30
こりす 30 num
page:0
雑誌 ＣＯＭＩＣポプリクラブ ２０１５年５月号
1 name_num
雑誌 ＣＯＭＩＣ漫華鏡 ２０１５年１月号
1 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１４年１０月号
1 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１４年９月号
1 name_num
雑誌 ＣＯＭＩＣ失楽天 ２０１４年６月号
1 name_num
しちゃってもいいよ こりす&nbsp&nbsp
11
雑誌 ＣＯＭＩＣポプリクラブ ２０１４年１月号
1 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１３年１２月号
1 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１３年１１月号
1 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１３年９月号
1 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１３年８月号
1 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１３年５月号
2 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１３年４月号
2 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１２年１１月号
2 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１２年９月号
1 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１２年７月号
1 name_num
ラブ・ペロ こりす&nbsp&nbsp
15
雑誌 ＣＯＭＩＣポプリクラブ ２０１２年５月号
2 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１２年３月号
1 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１２年２月号
1 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１２年１月号
1 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１１年９月号
1 name_num
雑誌 ＣＯＭＩＣポプリクラブ ２０１１年６月号
1 name_num
雑誌 コミックメガストアＨ ２０１０年１０月号
1 name_num
1 num
雑誌 二次元ドリームマガジン ２０１０年２月号
1 name_num
1 num
<type 'unicode'> <type 'unicode'>
<type 'unicode'> <type 'unicode'>
catch
<type 'unicode'> <type 'unicode'>
<type 

In [47]:
# -*- coding: utf-8 -*- 
import urllib , requests , sys ,string ,time
from bs4 import BeautifulSoup
from bs4 import SoupStrainer

fkey = open('key.txt', 'r')
key = fkey.readline()#key=作者
fkey.close()

#檢查BOM
if '%EF%BB%BF' in urllib.quote(key):
    print 'fuck ms'

'''
link = "http://comiclist.jp/index.php?p=s&mode=ss&keyword=" + urllib.quote(key) + "&type=title"
res = requests.get(link)
'''
#當sjis輸出utf8的url
key2 = urllib.quote(key.decode('utf8').encode('sjis'))
#utf8的url翻sjis
key3 = urllib.unquote(key2.decode('sjis').encode('utf8'))
#虎穴用sjis

#print key2
#print key3
adict = { 
'mode':'search',
'page_num':'0',
'search_cat':'',
'keyword':key
}
'''
res = requests.post("http://www.daito-i.com/top/show_unit.php", data = adict)

res.encoding =  res.apparent_encoding#亂碼處理
only_a_tags = SoupStrainer(id='contents')#縮小檢索範圍 id="wrapper"
'''
h1='''
<li>
<div class="itmBox">	<a href="magazine/detail.php?code=4910137600157">
	<img src="images/magazine/max/2014/4910137600157.jpg" alt="ＣＯＭＩＣ漫華鏡[２０１５年１月号]"></a><br />
<div class="marks"></div>
雑誌<br />
「<a href="magazine/detail.php?code=4910137600157">ＣＯＭＩＣ漫華鏡</a>
」<br />
<a href="show_unit.php?mode=search&disp_type=2&keyword=%EF%BC%92%EF%BC%90%EF%BC%91%EF%BC%95%E5%B9%B4%EF%BC%91%E6%9C%88%E5%8F%B7">２０１５年１月号</a>&nbsp&nbsp<br />
<span class="itmPrc">400円</span>
</div></li>
'''

h2='''
<html>
<body>
<li>
<div class="itmBox"><a href="detail.php?code=9784894656833">[詳細ページ]</a>
<img src="../images/printing.gif"><br><br />
<div class="marks"><img src="../img/new.gif" alt="new" align="absmidedle" border="0">&nbsp;</div>
予約商品<br />
「<a href="detail.php?code=9784894656833">ガールズ・マジョリティー</a>
」<br />
<a href="../show_unit.php?mode=search&disp_type=2&keyword=%E9%AB%98%E5%B2%A1%E5%9F%BA%E6%96%87">高岡基文</a>&nbsp&nbsp<br />
<span class="itmPrc">1,080円</span>
</div></li>
</body>
</html>
'''
soup = ''
sou = ''
#soup = BeautifulSoup(res.text ,"lxml" ,  parse_only=only_a_tags)#.prettify()
soup = BeautifulSoup(h1 ,"lxml")# ,  parse_only=only_a_tags)#.prettify()
sou = BeautifulSoup(h2 ,"lxml")
#print soup.select('.marks')[0].next_sibling[1:]
#print sou.select('.marks')[0].next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling
#print sou.a
#print soup.select('a')[1].text,soup.select('a')[2].text
#print sou.select('a')[1].text,sou.select('a')[2].text
#
#全轉半
def Q2B(ustring):
    fs='０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs='0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    for tm in ustring:
        if fs.find(tm)+1:
            tm = hs[fs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    return ustring
#半轉全
def B2Q(ustring):
    fs='０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs='0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    for tm in ustring:
        if hs.find(tm)+1:
            tm = fs[hs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    return ustring

def find_x(blink='http://www.daito-i.com/top/comics/detail.php?code=9784799207833'):
    bdate = ''
    res = requests.get(blink)
    res.encoding =  res.apparent_encoding
    only_a_tags = SoupStrainer(class_="rec")
    soup = BeautifulSoup(res.text ,"lxml",  parse_only=only_a_tags)
    print soup.select('strong')[0].text,soup.select('strong')[1].text#書名_作者
    #print len(soup.select('.goodTxt')),soup.select('.goodTxt')[0]#主內容
    
    nnum = str(soup.select('.goodTxt')[0]).count('<br/>')#換行出現次數
    for tm in soup.select('.goodTxt')[0].select('p'):
        #print tm.prettify()
        if (str(tm).count('<br/>') > 5):# and str(tm).count(key):
            print tm.text,type(tm.text)
            bdate = bdate + '(' + str(str(tm).count('<br/>')) + ')-'
            #tm=str(tm).replace('<br/>', '\n')
            tm = tm.get_text('\n-')
            print type(tm)
            bdate = bdate + tm + '%\n'
            '''
            while tm.find(u'【') > 0:
                if key.decode('utf8') in tm[tm.find(u'【')+1:tm.find(u'】')]:
                    bdate = bdate + tm[:tm.find(u'【')] + '_'
                tm=tm[tm.find(u'】')+1:]
            #'''
    #print 'find3'
    bdate = bdate[:bdate.find(u'%')+1]#去多餘
    return bdate,nnum
#print len(key.decode('utf8')) ,len(str(key)), find_3(),str(key)
cb,bb=find_x()
#print type(cb)
print cb
print '=',bb

if 'sa' in 'gfdsa':
    print  'ok'
print type(key)
KEY = B2Q(key)
KEY= KEY
print type(key), type(KEY)
print key,KEY

フリージング２８ 金光鉉／原作：林達永&nbsp&nbsp
第１９０話　拒絶Ｉ第１９１話　拒絶ＩＩ第１９２話　拒絶ＩＩＩ第１９３話　役割Ｉ第１９４話　役割ＩＩ第１９５話　価値第１９６話　相応の力 <type 'unicode'>
<type 'unicode'>
(6)-第１９０話　拒絶Ｉ
-第１９１話　拒絶ＩＩ
-第１９２話　拒絶ＩＩＩ
-第１９３話　役割Ｉ
-第１９４話　役割ＩＩ
-第１９５話　価値
-第１９６話　相応の力%
= 11
ok
<type 'str'>
<type 'str'> <type 'str'>
こりす こりす
